# Portfolio Composition Analysis

This notebook provides comprehensive analysis of your personal portfolio, including:

- Asset allocation breakdown
- Sector exposure analysis
- ETF look-through (where available)
- Current portfolio value and weights

**Configuration:** Edit `src/config_ticker.json` to define your portfolio holdings.


In [ ]:
import sys
import os

# Add src directory to path
sys.path.insert(0, os.path.abspath('../src'))

import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from core.portfolio import analyze_portfolio_composition
from core.etf_analyzer import analyze_portfolio_with_lookthrough, get_etf_info
from config import PORTFOLIO_FILE

pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)


## 1. Load Portfolio Configuration


In [ ]:
with open(f'../src/{PORTFOLIO_FILE}', 'r') as f:
    portfolio = json.load(f)

print("Current Portfolio Configuration:")
print("=" * 50)
for item in portfolio:
    print(f"{item['ticker']:6} - {item['units']:8.2f} units ({item['type']})")
print("=" * 50)


## 2. Asset Allocation Analysis


In [ ]:
# Generate asset and sector allocation visualizations
fig_asset, fig_sector = analyze_portfolio_composition(f'../src/{PORTFOLIO_FILE}')

if fig_asset:
    fig_asset.show()

if fig_sector:
    fig_sector.show()


## 3. Detailed Holdings Table


In [ ]:
import yfinance as yf

# Create detailed holdings table
holdings_data = []
total_value = 0

for item in portfolio:
    ticker = item['ticker']
    units = item['units']
    
    try:
        ticker_obj = yf.Ticker(ticker)
        price = ticker_obj.fast_info['lastPrice']
        market_value = units * price
        total_value += market_value
        
        info = ticker_obj.info
        sector = info.get('sector', 'N/A') if item['type'] == 'stock' else 'ETF'
        
        holdings_data.append({
            'Ticker': ticker,
            'Type': item['type'].upper(),
            'Units': units,
            'Price': price,
            'Market Value': market_value,
            'Sector': sector
        })
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

df_holdings = pd.DataFrame(holdings_data)
df_holdings['Weight (%)'] = (df_holdings['Market Value'] / total_value * 100)
df_holdings = df_holdings.sort_values('Weight (%)', ascending=False)

print(f"\nTotal Portfolio Value: ${total_value:,.2f}")
print("\nDetailed Holdings:")
display(df_holdings.style.format({
    'Units': '{:.2f}',
    'Price': '${:.2f}',
    'Market Value': '${:,.2f}',
    'Weight (%)': '{:.2f}%'
}))


## 4. ETF Look-Through Analysis

This section attempts to analyze the underlying holdings of ETFs in your portfolio to understand indirect exposure.


In [ ]:
# Analyze portfolio with ETF look-through
lookthrough_df = analyze_portfolio_with_lookthrough(portfolio, max_etf_holdings=15)

if not lookthrough_df.empty:
    print("\nTotal Exposure (Direct + Indirect via ETFs):")
    print("=" * 70)
    
    display(lookthrough_df.style.format({
        'Exposure_Value': '${:,.2f}',
        'Portfolio_Weight': '{:.2%}'
    }))
    
    # Visualize combined exposure
    fig = px.bar(
        lookthrough_df.head(15),
        x='Ticker',
        y='Portfolio_Weight',
        title='Total Exposure Including ETF Look-Through (Top 15)',
        labels={'Portfolio_Weight': 'Weight (%)', 'Ticker': 'Asset'}
    )
    fig.update_yaxes(tickformat='.1%')
    fig.show()
else:
    print("ETF holdings data not available for look-through analysis.")


## 5. ETF Details

Detailed information about each ETF in the portfolio.


In [ ]:
# Get details for all ETFs in portfolio
etf_items = [item for item in portfolio if item.get('type') == 'etf']

for etf_item in etf_items:
    ticker = etf_item['ticker']
    print(f"\n{'='*70}")
    print(f"ETF: {ticker}")
    print(f"{'='*70}")
    
    etf_info = get_etf_info(ticker)
    
    for key, value in etf_info.items():
        print(f"{key.replace('_', ' '):.<30} {value}")


## 6. Sector Breakdown

Comprehensive sector exposure across all holdings (excluding ETFs).


In [ ]:
# Sector breakdown for stocks only
stock_holdings = df_holdings[df_holdings['Type'] == 'STOCK'].copy()

if not stock_holdings.empty:
    sector_summary = stock_holdings.groupby('Sector').agg({
        'Market Value': 'sum',
        'Ticker': 'count'
    }).reset_index()
    sector_summary.columns = ['Sector', 'Market Value', 'Count']
    sector_summary['Weight (%)'] = (sector_summary['Market Value'] / stock_holdings['Market Value'].sum() * 100)
    sector_summary = sector_summary.sort_values('Weight (%)', ascending=False)
    
    print("\nSector Exposure (Stocks Only):")
    display(sector_summary.style.format({
        'Market Value': '${:,.2f}',
        'Weight (%)': '{:.2f}%'
    }))


## Summary

This notebook provides a comprehensive view of your portfolio composition. Key insights:

1. **Asset Allocation**: Shows how your capital is distributed across different assets
2. **Sector Exposure**: Identifies concentration in specific sectors
3. **ETF Look-Through**: Reveals indirect exposure through ETF holdings

**Next Steps:**

- Adjust holdings in `config_ticker.json` to rebalance
- Review sector concentration for diversification
- Use the Index Simulation notebook to analyze market scenarios

